In [1]:
import pandas as pd

In [2]:
states = pd.read_csv("ECMO_data_clustered.csv")

In [3]:
actions = pd.read_csv("ECMO_action_space.csv").drop(columns="csn")

In [8]:
merged_df = pd.concat([states,actions],axis=1)

In [11]:
merged_df

,temperature,map_line,map_cuff,pulse,unassisted_resp_rate,end_tidal_co2,o2_flow_rate,base_excess,bicarb_(hco3),blood_urea_nitrogen_(bun),...,sweep_range_1,sweep_range_2,sweep_range_3,sweep_range_4,flow_range_0,flow_range_1,flow_range_2,flow_range_3,flow_range_4,action_number
0,1.654985,1.543293,0.712217,1.037490,-0.277493,1.443907,-0.794470,-0.319874,-0.824496,0.070542,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1419
1,0.179388,0.036551,0.685907,-0.148586,-0.277493,1.223377,-0.794470,-0.319874,-0.880290,0.051429,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1924
2,-1.182702,0.325409,0.685907,-0.125951,-0.277493,1.100080,-0.794470,-0.319874,-0.746384,0.032315,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1924
3,-0.501657,-0.420155,-0.300701,1.405686,-0.312486,0.691097,-0.794470,-0.319874,-1.023762,0.032315,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1925
4,1.314463,-0.100070,0.093942,3.125949,-0.067534,0.817400,-0.794470,-0.319874,-0.387706,0.032315,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15187,-0.501657,-0.724626,-0.234927,-0.495657,0.632326,-1.483133,1.449991,-0.161775,-0.827684,-0.655770,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,724
15188,-1.182702,-1.524839,-0.234927,-0.359847,0.562340,-1.483133,1.449991,-0.161775,-0.827684,-0.770451,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,724
15189,-0.501657,-1.408711,-0.234927,-0.300116,0.637325,-1.483133,1.449991,-0.161775,-0.827684,-0.770451,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,724
15190,-0.643542,-0.603619,-0.234927,0.359444,-0.090863,-1.483133,1.449991,-0.161775,-0.827684,-0.770451,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,725


In [57]:
# Assuming `merged_df` is your dataframe
# Define groups of columns for each action
action_groups = {
    'fio2': ['fio2_range_0', 'fio2_range_1', 'fio2_range_2', 'fio2_range_3'],
    'peep': ['peep_range_0', 'peep_range_1', 'peep_range_2', 'peep_range_3', 'peep_range_4'],
    'vent_rate_set': ['vent_rate_set_range_0', 'vent_rate_set_range_1', 'vent_rate_set_range_2', 'vent_rate_set_range_3'],
    'sweep': ['sweep_range_0', 'sweep_range_1', 'sweep_range_2', 'sweep_range_3', 'sweep_range_4'],
    'flow': ['flow_range_0', 'flow_range_1', 'flow_range_2', 'flow_range_3', 'flow_range_4']
}

# Create a new dataframe to store the non-one-hot encoded actions
actions_df = pd.DataFrame()

# Convert one-hot encoded groups to single action columns
for action, columns in action_groups.items():
    actions_df[action] = merged_df[columns].idxmax(axis=1).str.extract(r'_(\d+)$').astype(int)

# Combine back with the original dataframe if needed
merged_df_transformed = pd.concat([merged_df.drop(columns=sum(action_groups.values(), [])), actions_df], axis=1)

In [58]:
# Specify the desired order for the action columns
action_order = ['vent_rate_set', 'peep', 'fio2', 'flow', 'sweep']

# Identify other columns (non-action columns)
non_action_columns = [col for col in merged_df_transformed.columns if col not in action_order]

# Combine non-action columns with the reordered action columns
new_column_order = non_action_columns + action_order

# Reorder the DataFrame columns
merged_df_transformed = merged_df_transformed[new_column_order]

In [59]:
# Read the unique CSN values from the specified CSV file
train_data = pd.read_csv("../Continuous Data/test_data_continuous_Pos_for_Survival.csv")
csv_csn_values = train_data.csn.unique()
# Filter the `merged_df_transformed` DataFrame to keep only rows with matching CSN values
train_data_continuous_no_R_for_Survival_Discrete = merged_df_transformed[merged_df_transformed['csn'].isin(csv_csn_values)]

# Ensure the indexes align before concatenating
train_data_continuous_no_R_for_Survival_Discrete = train_data_continuous_no_R_for_Survival_Discrete.reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

# Concatenate the `reward` column from `train_data` to the filtered DataFrame
train_data_continuous_no_R_for_Survival_Discrete = pd.concat(
    [train_data_continuous_no_R_for_Survival_Discrete, train_data[['reward']]], axis=1
)

In [60]:
train_data_continuous_no_R_for_Survival_Discrete

,temperature,map_line,map_cuff,pulse,unassisted_resp_rate,end_tidal_co2,o2_flow_rate,base_excess,bicarb_(hco3),blood_urea_nitrogen_(bun),...,rass_score,CAM_ICU,csn,action_number,vent_rate_set,peep,fio2,flow,sweep,reward
0,-0.388150,0.633783,0.488585,-1.733039,-0.598262,0.642981,-0.960726,-0.066744,0.138355,0.376358,...,3.363714,1.311655,4510511039,827,1,3,1,1,0,0.100000
1,-0.842180,0.398571,0.488585,-1.643757,-0.487451,1.100080,-0.960726,-0.066744,0.310521,0.376358,...,1.392030,1.311655,4510511039,827,1,3,1,1,0,0.278125
2,-0.728672,-0.135201,0.488585,-1.813016,-0.137520,1.130153,-0.960726,-0.066744,0.238785,0.376358,...,0.124519,0.781225,4510511039,927,1,4,1,1,0,0.362500
3,-0.671919,0.481548,0.488585,-1.936755,-0.487451,1.070008,-0.960726,-0.066744,0.128791,0.376358,...,-0.157150,-0.810065,4510511039,927,1,4,1,1,0,0.400000
4,-0.388150,-0.517742,0.488585,-1.751902,-0.114192,1.391181,-0.960726,-0.066744,-0.143805,0.051429,...,-0.016316,-0.810065,4510511039,951,2,4,1,0,0,0.593750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3379,1.087448,-0.623136,-0.037606,1.828207,0.387375,-0.472703,-0.628213,0.397329,0.702675,-0.617543,...,1.251195,-0.810065,70454151283,1,0,0,0,0,0,0.980000
3380,0.803679,0.013131,-0.037606,1.164246,0.212410,-0.472703,-0.628213,0.397329,0.702675,-0.617543,...,0.969526,-0.810065,70454151283,1,0,0,0,0,0,0.985000
3381,0.917187,-0.291341,-0.037606,1.383051,0.527347,-0.472703,-0.628213,0.397329,0.893970,-0.923358,...,0.969526,-0.810065,70454151283,1,0,0,0,0,0,0.990000
3382,0.349649,-0.634846,-0.037606,0.555615,-0.067534,-0.472703,-0.628213,0.397329,1.467855,-0.923358,...,0.687857,-0.810065,70454151283,1,0,0,0,0,0,0.995000


In [61]:
# train_data_continuous_no_R_for_Survival_Discrete.drop(columns='action_number').to_csv("train_data_continuous_no_R_for_Survival_Discrete.csv",index=False)
train_data_continuous_no_R_for_Survival_Discrete.drop(columns='action_number').to_csv("test_data_discrete_Pos_for_Survival.csv",index=False)